In [ ]:
using Revise
using Quantics
import Quantics: DiscretizedGrid
using Plots
import TensorCrossInterpolation as TCI

using ITensors
ITensors.disable_warn_order()

## Differrential equation

$$
\frac{d f(t)}{d t} = - a f(t),
$$
where $0 \le t < t_\mathrm{max}$, $a > 0$ with the boundary condition $f(0) = 1$.

The solution is 

$$
f(t) = e^{- a t}.
$$

In QTT, we solve the equation for $g(t) \equiv \frac{d f(t)}{d t}$.

In [ ]:
R = 16
tmax = 10.0
a = 2.0
f0 = 1.0

grid = DiscretizedGrid{1}(R, (0.0,), (tmax,))
times = collect(LinRange(0.0, tmax, 2^R+1)[1:end-1])
sites = [Index(2, "Qubit,q=$q") for q in 1:R]

In [ ]:
integrator = (tmax/2.0^R) * Quantics.upper_lower_triangle_matrix(sites, 1.0; upper_or_lower=:lower)

## First check exact solution

In [ ]:
gmps = -a * Quantics.expqtt(sites, - a * tmax)

In [ ]:
gt = vec(Array(reduce(*, gmps), reverse(sites)))

#p = plot(yaxis=:log)
p = plot()
plot!(p, times, gt, marker=:x)
plot!(p, times, -a .* exp.(-a .* times))

In [ ]:
fmps = apply(integrator, gmps) + f0 * Quantics.onemps(Float64, sites)

In [ ]:
ft = vec(Array(reduce(*, fmps), reverse(sites)))

In [ ]:
p = plot()
plot!(p, times, ft)
plot!(p, times, exp.(-a .* times))

## Iterative solution

In [ ]:
# Compute g(t) from g(t)
function new_gt(gmps::MPS; kwargs...)::MPS
    fmps = apply(integrator, gmps; kwargs...) + f0 * Quantics.onemps(Float64, sites)
    return -a * fmps
end

In [ ]:
gmps = 0.0 * Quantics.onemps(Float64, sites) # 0 everywhere

mix = 0.5
for iter in 1:20
    bondim = maximum(dim.(linkinds(gmps)))
    println("iter= $(iter), D= $(bondim)")
    gmps = mix * new_gt(gmps; cutoff=1e-20) + (1-mix) * gmps
end

In [ ]:
gt = vec(Array(reduce(*, gmps), reverse(sites)))

p = plot()
plot!(p, times, gt, marker=:x)
plot!(p, times, -a .* exp.(-a .* times))